Reference:
http://csc.lsu.edu/~jianhua/FPGrowth.pdf
https://adataanalyst.com/machine-learning/fp-growth-algorithm-python-3/


In [1]:
import datetime as dt
import pandas as pd
import numpy as np
import random as rand
import operator
import csv
import time
import copy
import collections
from itertools import combinations

In [2]:
file = r'E:\School\@Grad School\Data Mining\Project 1\adult.data'
#file.read()

df = pd.read_csv(file, sep=',',
                 names=["Age", "Workclass", "fnlwgt","education", "education_num","marital status","occupation","relationship","Race","Sex","capital_gain","capital_loss","hours_per_week","native country","income"],
                 skipinitialspace=True)
#df.replace(to_replace='?', value=np.nan)
df.head()


,Age,Workclass,fnlwgt,education,education_num,marital status,occupation,relationship,Race,Sex,capital_gain,capital_loss,hours_per_week,native country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [3]:
def clean_data(df):
    df= df.applymap(lambda x: np.nan if x == '?' else x)
    df=df.dropna(axis=0)
    df.drop('fnlwgt', axis=1, inplace=True)
    df.drop('education_num', axis=1, inplace=True)
    df['Age'] = pd.cut(df['Age'], [0, 26, 46, 66,100], 
                        labels = ['Young', 'Middle-aged', 'Senior', 'Old'], 
                        right = True, include_lowest = True)
    df['hours_per_week'] = pd.cut(df['hours_per_week'], [0, 25, 40, 60, 100], 
                              labels = ['Part-time', 'Full-time', 'Over-time', 'Too-much'], 
                              right = True, include_lowest = True)
    df['capital_gain'] = pd.cut(df['capital_gain'], [0, 1, 10000000], 
                           labels = ['No-Gain', 'Positive-Gain'], 
                           right = True, include_lowest = True)
    df['capital_loss'] = pd.cut(df['capital_loss'], [0, 1, 10000000],
                            labels = ['No-Loss', 'Positive-Loss'], 
                            right = True, include_lowest = True)


    return df

In [4]:
dataset = [['M', 'O', 'N', 'K', 'E', 'Y'], 
            ['D', 'O', 'N', 'K', 'E', 'Y'], 
            ['M', 'A', 'K', 'E'],
            ['M', 'U', 'C', 'K', 'Y'],
            ['C', 'O', 'O', 'K', 'I', 'E']]

def open_data():
    
    
    data = clean_data(df)
    dataset= data.values.tolist()
    return dataset
dataset= open_data()
dataset

[['Middle-aged',
  'State-gov',
  'Bachelors',
  'Never-married',
  'Adm-clerical',
  'Not-in-family',
  'White',
  'Male',
  'Positive-Gain',
  'No-Loss',
  'Full-time',
  'United-States',
  '<=50K'],
 ['Senior',
  'Self-emp-not-inc',
  'Bachelors',
  'Married-civ-spouse',
  'Exec-managerial',
  'Husband',
  'White',
  'Male',
  'No-Gain',
  'No-Loss',
  'Part-time',
  'United-States',
  '<=50K'],
 ['Middle-aged',
  'Private',
  'HS-grad',
  'Divorced',
  'Handlers-cleaners',
  'Not-in-family',
  'White',
  'Male',
  'No-Gain',
  'No-Loss',
  'Full-time',
  'United-States',
  '<=50K'],
 ['Senior',
  'Private',
  '11th',
  'Married-civ-spouse',
  'Handlers-cleaners',
  'Husband',
  'Black',
  'Male',
  'No-Gain',
  'No-Loss',
  'Full-time',
  'United-States',
  '<=50K'],
 ['Middle-aged',
  'Private',
  'Bachelors',
  'Married-civ-spouse',
  'Prof-specialty',
  'Wife',
  'Black',
  'Female',
  'No-Gain',
  'No-Loss',
  'Full-time',
  'Cuba',
  '<=50K'],
 ['Middle-aged',
  'Private',
  '

In [5]:
#scan data and find support for each item
def Scan_D(dataset):
    C1 = {}
    for item in dataset:
        for itemset in item:
            if itemset in C1:
                C1[itemset] += 1
            elif itemset not in C1:
                C1[itemset] = 1
    return C1
C1=Scan_D(open_data())
C1


{'Middle-aged': 15721,
 'State-gov': 1279,
 'Bachelors': 5044,
 'Never-married': 9726,
 'Adm-clerical': 3721,
 'Not-in-family': 7726,
 'White': 25933,
 'Male': 20380,
 'Positive-Gain': 2538,
 'No-Loss': 28735,
 'Full-time': 17704,
 'United-States': 27504,
 '<=50K': 22654,
 'Senior': 7299,
 'Self-emp-not-inc': 2499,
 'Married-civ-spouse': 14065,
 'Exec-managerial': 3992,
 'Husband': 12463,
 'No-Gain': 27624,
 'Part-time': 3261,
 'Private': 22286,
 'HS-grad': 9840,
 'Divorced': 4214,
 'Handlers-cleaners': 1350,
 '11th': 1048,
 'Black': 2817,
 'Prof-specialty': 4038,
 'Wife': 1406,
 'Female': 9782,
 'Cuba': 92,
 'Masters': 1627,
 '9th': 455,
 'Married-spouse-absent': 370,
 'Other-service': 3212,
 'Jamaica': 80,
 'Over-time': 8145,
 '>50K': 7508,
 'Some-college': 6678,
 'Too-much': 1052,
 'Asian-Pac-Islander': 895,
 'India': 100,
 'Young': 6413,
 'Own-child': 4466,
 'Assoc-acdm': 1008,
 'Sales': 3584,
 '7th-8th': 557,
 'Transport-moving': 1572,
 'Amer-Indian-Eskimo': 286,
 'Mexico': 610,
 

In [6]:
#discard infrequent item
def frequent (min_sup):
    frequent = {k:v for k, v in C1.items() if v >= min_sup*len(dataset)}
    return frequent
frequent(0.6)

{'White': 25933,
 'Male': 20380,
 'No-Loss': 28735,
 'United-States': 27504,
 '<=50K': 22654,
 'No-Gain': 27624,
 'Private': 22286}

In [7]:
def initset (dataset):
    initial ={}
    for trans in dataset:
        initial[tuple(trans)] =1
    #print (initial)
    return initial
initset(dataset)

{('Middle-aged',
  'State-gov',
  'Bachelors',
  'Never-married',
  'Adm-clerical',
  'Not-in-family',
  'White',
  'Male',
  'Positive-Gain',
  'No-Loss',
  'Full-time',
  'United-States',
  '<=50K'): 1,
 ('Senior',
  'Self-emp-not-inc',
  'Bachelors',
  'Married-civ-spouse',
  'Exec-managerial',
  'Husband',
  'White',
  'Male',
  'No-Gain',
  'No-Loss',
  'Part-time',
  'United-States',
  '<=50K'): 1,
 ('Middle-aged',
  'Private',
  'HS-grad',
  'Divorced',
  'Handlers-cleaners',
  'Not-in-family',
  'White',
  'Male',
  'No-Gain',
  'No-Loss',
  'Full-time',
  'United-States',
  '<=50K'): 1,
 ('Senior',
  'Private',
  '11th',
  'Married-civ-spouse',
  'Handlers-cleaners',
  'Husband',
  'Black',
  'Male',
  'No-Gain',
  'No-Loss',
  'Full-time',
  'United-States',
  '<=50K'): 1,
 ('Middle-aged',
  'Private',
  'Bachelors',
  'Married-civ-spouse',
  'Prof-specialty',
  'Wife',
  'Black',
  'Female',
  'No-Gain',
  'No-Loss',
  'Full-time',
  'Cuba',
  '<=50K'): 1,
 ('Middle-aged',
 

In [8]:
#sort the frequent items in descending order based on support count
def sort_freq(dataset, min_sup=0.6):
    freq = frequent(min_sup)
    freq_set = set(freq.keys())
    sorted_freq = [] 
    for trans in dataset:
        freq_trans = {}
        for item in trans:
            if item in freq_set:
                freq_trans[item] = freq[item]
        if len(freq_trans) > 0:
            sort_freq = [val[0] for val in sorted(freq_trans.items(), key=lambda x: x[1], reverse=True)]
            sorted_freq.append(sort_freq)
    
    return sorted_freq
#sort_freq(dataset)


In [9]:
class tree:
    def __init__ (self, value, freq, parents):
        self.name = value
        self.count = freq
        self.node_link = None
        self.parent = parents
        self.children = {}
        
        
    
    #increment the count
    def inc_count (self, count):
        self.count += count
        

In [10]:
#FP tree
def make_tree(dataset, min_sup=.6):
    
    #first scan  
    freq = frequent(min_sup)    
    freq_set = set(freq.keys())
    if len(freq_set) == 0:
        return None, None
    
    for f in freq:
        freq[f]= [freq[f], None]
    
    root = tree('Null', 1, None)
    #i=0
    for transaction, count in dataset.items():       
        freq_trans = {}  
       
        for itemset in transaction:
            if itemset in freq_set:
                freq_trans[itemset] = freq[itemset][0]
            
        if len(freq_trans) > 0:
            sort_freq = [val[0] for val in sorted(freq_trans.items(), key=lambda x:x[1], reverse=True)]
            get_path(sort_freq, root, freq, count)
            #i+=1
    return root, freq

In [11]:
def get_path(itemset, root, frequent, count):

    if itemset[0] not in root.children:
        root.children[itemset[0]] = tree(itemset[0], count, root)
        
        if frequent[itemset[0]][1] == None:
            frequent[itemset[0]][1] = root.children[itemset[0]]
        else:
            update_nodelink(frequent[itemset[0]][1], root.children[itemset[0]])

    else:
        root.children[itemset[0]].inc_count(count)
    if len(itemset) >1:
        get_path(itemset[1::], root.children[itemset[0]], frequent, count)

In [12]:
def update_nodelink(test_node, target_node):
    while (test_node.node_link != None):
        test_node = test_node.node_link
    test_node.node_link = target_node

In [13]:
#FPtree, count = make_tree(init, 0.6)

#myHeaderTab
#FPtree.display()
#count

In [14]:
def go_up(leaf, prefix): #ascends from leaf node to root
    if leaf.parent != None:
        prefix.append(leaf.name)
        go_up(leaf.parent, prefix)

In [15]:
def find_prefix(pattern_base, tree): 
    #node = tree[pattern_base][1][1]
    cond_pattern = {}
    while tree != None:
        prefix = []
        go_up(tree, prefix)
        if len(prefix) > 1: 
            cond_pattern[tuple(prefix[1:])] = tree.count
        tree = tree.node_link
    return cond_pattern

In [16]:
#find_prefix('Y',count['Y'][1])

In [17]:
def condition_FPtree(FPtree, freq, min_sup,prefix, freq_set):
    L=[v[0] for v in sorted(freq.items(),key=lambda x: x[1])]
    #=print( L)
    
    for pattern_base in L:
        new_freqset = prefix.copy()
        new_freqset.add(pattern_base)
        #add frequent itemset to final list of frequent itemsets
        freq_set.append(new_freqset)
        conditional_pattern_base = find_prefix(pattern_base, freq[pattern_base][1])
        #print(conditional_pattern_base)
        
        if(conditional_pattern_base!={}):
            conditional_FPtree, conditional_freq = make_tree(conditional_pattern_base, min_sup)
        #conditional_FPtree.display()
        
            if conditional_freq != None:
                condition_FPtree(conditional_FPtree, conditional_freq, min_sup, new_freqset, freq_set)


In [18]:
init= initset(dataset)
s = time.time()
FPtree, count = make_tree(init)
freq_set=[]
condition_FPtree(FPtree, count, 0.1,set([]), freq_set)
e = time.time()
r=e-s
r

0.21094417572021484

In [19]:
freq_set 

[{'Male'},
 {'Male', 'Other-service'},
 {'Male', 'Unmarried'},
 {'Male', 'Part-time'},
 {'Male', 'Sales'},
 {'Adm-clerical', 'Male'},
 {'Exec-managerial', 'Male'},
 {'Craft-repair', 'Male'},
 {'Male', 'Prof-specialty'},
 {'Divorced', 'Male'},
 {'Male', 'Own-child'},
 {'Bachelors', 'Male'},
 {'Male', 'Young'},
 {'Male', 'Some-college'},
 {'Male', 'Senior'},
 {'>50K', 'Male'},
 {'Male', 'Not-in-family'},
 {'Male', 'Over-time'},
 {'Male', 'Never-married'},
 {'Female', 'Male'},
 {'HS-grad', 'Male'},
 {'Husband', 'Male'},
 {'Male', 'Married-civ-spouse'},
 {'Male', 'Middle-aged'},
 {'Full-time', 'Male'},
 {'Male'},
 {'Male', 'Private'},
 {'Male', 'Other-service', 'Private'},
 {'Male', 'Private', 'Unmarried'},
 {'Male', 'Part-time', 'Private'},
 {'Male', 'Private', 'Sales'},
 {'Adm-clerical', 'Male', 'Private'},
 {'Exec-managerial', 'Male', 'Private'},
 {'Craft-repair', 'Male', 'Private'},
 {'Male', 'Private', 'Prof-specialty'},
 {'Divorced', 'Male', 'Private'},
 {'Male', 'Own-child', 'Privat